<a href="https://colab.research.google.com/github/AriyaArKa/2-2-Labs/blob/main/Faltu_Niyom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import os

print("Upload your kaggle.json file")
uploaded = files.upload()  # Upload kaggle.json

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c fragments-of-feeling
!unzip -q fragments-of-feeling.zip -d data
!rm -rf fragments-of-feeling.zip

Upload your kaggle.json file


Saving kaggle.json to kaggle.json
  0% 0.00/1.14M [00:00<?, ?B/s]
100% 1.14M/1.14M [00:00<00:00, 781MB/s]


In [ ]:
import pandas as pd

train_df = pd.read_csv('data/train_emotions.csv')

print(train_df.head())

print(train_df['emotion_final'].value_counts())

  sentence_id  item_id                 title  \
0   1008068_1  1008068  Scared of my partner   
1   1008068_2  1008068  Scared of my partner   
2   1008068_3  1008068  Scared of my partner   
3   1008068_4  1008068  Scared of my partner   
4   1008068_5  1008068  Scared of my partner   

                                            sentence  upvotes           date  \
0  I feel like I am walking on eggshells, and whe...     12.0  1672534044000   
1  The thing is, he has never been violent toward...     12.0  1672534044000   
2  But I can't describe it, his depression turns ...     12.0  1672534044000   
3  Given that he has never been violent towards u...     12.0  1672534044000   
4                Or should I be listening to my gut?     12.0  1672534044000   

   emotion_final  emotion_int  
0  worthlessness            4  
1  worthlessness            4  
2   hopelessness            1  
3  worthlessness            4  
4   hopelessness            1  
emotion_final
sadness              695

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

X = train_df['sentence']
y = train_df['emotion_int']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert text to TF-IDF features
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)

In [ ]:
# Train a Logistic Regression model
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_tfidf, y_train)

# Predict on validation set
y_pred = model.predict(X_val_tfidf)

# Evaluate using macro F1-score
macro_f1 = f1_score(y_val, y_pred, average='macro')
print(f'Macro F1-score on validation set: {macro_f1:.4f}')

Macro F1-score on validation set: 0.3597


In [ ]:
# Load the test data
test_df = pd.read_csv('data/val_emotions_no_labels.csv')

# Transform test sentences to TF-IDF features
X_test_tfidf = vectorizer.transform(test_df['sentence'])

# Predict emotions
test_predictions = model.predict(X_test_tfidf)

# Create submission DataFrame
submission_df = pd.DataFrame({
    'sentence_id': test_df['sentence_id'],
    'predicted_emotion_int': test_predictions
})

# Save submission to CSV
submission_df.to_csv('submission.csv', index=False)
print("Submission file created: submission.csv")

Submission file created: submission.csv


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

# Load training data
train_df = pd.read_csv('data/train_emotions.csv')

# Prepare features and labels
X = train_df['sentence']
y = train_df['emotion_int']

# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)

# Train Logistic Regression model
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_tfidf, y_train)

# Evaluate on validation set
y_pred = model.predict(X_val_tfidf)
macro_f1 = f1_score(y_val, y_pred, average='macro')
print(f'Macro F1-score on validation set: {macro_f1:.4f}')

Macro F1-score on validation set: 0.3597


In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
from google.colab import files

# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Load training and test data
train_df = pd.read_csv('data/train_emotions.csv')
test_df = pd.read_csv('data/val_emotions_no_labels.csv')

# Combine title and sentence
train_df['text'] = train_df['title'].fillna('') + ' ' + train_df['sentence']
test_df['text'] = test_df['title'].fillna('') + ' ' + test_df['sentence']

# Prepare features and labels
X = train_df['text']
y = train_df['emotion_int']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Custom Dataset class
class EmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels if labels is not None else [0] * len(texts)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts.iloc[idx])
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        item = {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }
        if self.labels is not None and not isinstance(self.labels, list):
            item['labels'] = torch.tensor(self.labels.iloc[idx], dtype=torch.long)
        elif isinstance(self.labels, list):
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float).to('cuda')

# Custom model with weighted loss
class WeightedBertForSequenceClassification(BertForSequenceClassification):
    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = super().forward(input_ids, attention_mask=attention_mask, labels=labels)
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
            loss = loss_fct(outputs.logits, labels)
            return (loss, outputs.logits)
        return outputs

model = WeightedBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=8)

# Prepare datasets
train_dataset = EmotionDataset(X_train, y_train, tokenizer)
val_dataset = EmotionDataset(X_val, y_val, tokenizer)
test_dataset = EmotionDataset(test_df['text'], None, tokenizer)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    run_name='bert_emotion_classification',
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='macro_f1',
    greater_is_better=True,
    logging_strategy='epoch',
    fp16=True,
    warmup_steps=500,
    weight_decay=0.1,
    learning_rate=2e-5
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=lambda p: {'macro_f1': f1_score(p.label_ids, p.predictions.argmax(-1), average='macro')}
)

# Train the model
trainer.train()

# Error analysis
val_predictions = trainer.predict(val_dataset).predictions.argmax(-1)
from sklearn.metrics import classification_report
print("Validation Set Classification Report:")
print(classification_report(y_val, val_predictions, target_names=['sadness', 'hopelessness', 'loneliness', 'anger', 'worthlessness', 'suicide intent', 'emptiness', 'brain dysfunction']))

# Predict on test set
test_predictions = trainer.predict(test_dataset).predictions.argmax(-1)

# Create submission
submission_df = pd.DataFrame({
    'sentence_id': test_df['sentence_id'],
    'predicted_emotion_int': test_predictions
})
submission_df.to_csv('submission_bert.csv', index=False)
print("Submission file created: submission_bert.csv")
files.download('submission_bert.csv')

Some weights of WeightedBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Macro F1
1,1.484500,1.128807,0.558523
2,0.862000,1.098303,0.611319
3,0.496300,1.259117,0.628477
4,0.287400,1.369480,0.637216


Validation Set Classification Report:
                   precision    recall  f1-score   support

          sadness       0.69      0.67      0.68      1422
     hopelessness       0.61      0.57      0.59       938
       loneliness       0.73      0.76      0.74       713
            anger       0.72      0.74      0.73       605
    worthlessness       0.55      0.55      0.55       305
   suicide intent       0.64      0.70      0.67       266
        emptiness       0.46      0.54      0.50       207
brain dysfunction       0.62      0.67      0.64       108

         accuracy                           0.66      4564
        macro avg       0.63      0.65      0.64      4564
     weighted avg       0.66      0.66      0.66      4564



Submission file created: submission_bert.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Sample Given**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


# Load training and test data
train_df = pd.read_csv('data/train_emotions.csv')
val_df = pd.read_csv('data/val_emotions_no_labels.csv')



MODEL_NAME = 'google-bert/bert-base-uncased'

# The target labels are in 'emotion_int'. We need to create mappings for the model.
id2label = dict(enumerate(train_df['emotion_final'].astype('category').cat.categories))
label2id = {v: k for k, v in id2label.items()}
num_labels = len(id2label)

train_df = train_df.rename(columns={'emotion_int': 'label'})

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class EmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer(
            text, truncation=True, padding='max_length', max_length=self.max_len, return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df['sentence'].tolist(),
    train_df['label'].tolist(),
    test_size=0.1, # 10% for validation
    random_state=42,
    stratify=train_df['label'].tolist() # Ensure same emotion distribution in both sets
)

train_dataset = EmotionDataset(train_texts, train_labels, tokenizer)
eval_dataset = EmotionDataset(val_texts, val_labels, tokenizer)




def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average="macro")
    return {"f1_macro": f1}

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=num_labels, id2label=id2label, label2id=label2id
)



training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    report_to="none",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

print("\nStarting model training with evaluation...")
trainer.train()
print("Model training complete!")



final_val_texts = val_df['sentence'].tolist()
final_val_dataset = EmotionDataset(
    texts=final_val_texts,
    labels=[0] * len(final_val_texts), # Dummy labels
    tokenizer=tokenizer
)

print("\nGenerating predictions on the competition validation set...")
predictions = trainer.predict(final_val_dataset)

predicted_class_ids = np.argmax(predictions.predictions, axis=1)

submission_df = pd.DataFrame({
    'sentence_id': val_df['sentence_id'],
    'predicted_emotion_int': predicted_class_ids
})

submission_df.to_csv('submission.csv', index=False)

print("\nSubmission file created successfully!")
print("Submission File Head:")
print(submission_df.head())


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Starting model training with evaluation...


Epoch,Training Loss,Validation Loss,F1 Macro
1,1.172800,1.198894,0.534321


Model training complete!

Generating predictions on the competition validation set...



Submission file created successfully!
Submission File Head:
  sentence_id  predicted_emotion_int
0   1008eah_1                      0
1   1008eah_2                      0
2   1008eah_3                      0
3   1008eah_4                      0
4   100krou_1                      4


# Callback with patience 3

In [ ]:
import pandas as pd
import numpy as np
import torch
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers.trainer_callback import TrainerCallback, TrainerControl, TrainerState
from torch.utils.data import Dataset
from google.colab import files

# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Load training and test data
train_df = pd.read_csv('data/train_emotions.csv')
test_df = pd.read_csv('data/val_emotions_no_labels.csv')

# Combine title and sentence
train_df['text'] = train_df['title'].fillna('') + ' ' + train_df['sentence']
test_df['text'] = test_df['title'].fillna('') + ' ' + test_df['sentence']

# Prepare features and labels
X = train_df['text']
y = train_df['emotion_int']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Custom Dataset class
class EmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels if labels is not None else [0] * len(texts)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts.iloc[idx])
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        item = {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }
        if self.labels is not None and not isinstance(self.labels, list):
            item['labels'] = torch.tensor(self.labels.iloc[idx], dtype=torch.long)
        elif isinstance(self.labels, list):
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float).to('cuda')

# Custom model with weighted loss
class WeightedBertForSequenceClassification(BertForSequenceClassification):
    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = super().forward(input_ids, attention_mask=attention_mask, labels=labels)
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
            loss = loss_fct(outputs.logits, labels)
            return (loss, outputs.logits)
        return outputs

model = WeightedBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=8)

# Prepare datasets
train_dataset = EmotionDataset(X_train, y_train, tokenizer)
val_dataset = EmotionDataset(X_val, y_val, tokenizer)
test_dataset = EmotionDataset(test_df['text'], None, tokenizer)

# Custom callback for early stopping based on macro F1
class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, patience=3, threshold=0.0):
        self.patience = patience
        self.threshold = threshold
        self.best_macro_f1 = -float('inf')
        self.epochs_no_improve = 0

    def on_evaluate(self, args, state: TrainerState, control: TrainerControl, metrics, **kwargs):
        current_macro_f1 = metrics.get('eval_macro_f1', -float('inf'))
        if current_macro_f1 > self.best_macro_f1 + self.threshold:
            self.best_macro_f1 = current_macro_f1
            self.epochs_no_improve = 0
        else:
            self.epochs_no_improve += 1
            if self.epochs_no_improve >= self.patience:
                control.should_training_stop = True
                print(f"Early stopping triggered after {state.epoch} epochs with no improvement in macro F1.")

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    run_name='bert_emotion_classification',
    num_train_epochs=100,  # Set max epochs to 100
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='macro_f1',
    greater_is_better=True,
    logging_strategy='epoch',
    fp16=True,
    warmup_steps=500,
    weight_decay=0.1,
    learning_rate=2e-5
)

# Trainer with early stopping callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=lambda p: {'macro_f1': f1_score(p.label_ids, p.predictions.argmax(-1), average='macro')},
    callbacks=[EarlyStoppingCallback(patience=3, threshold=0.0)]  # Stop if no improvement for 3 epochs
)

# Train the model
trainer.train()

# Error analysis
val_predictions = trainer.predict(val_dataset).predictions.argmax(-1)
from sklearn.metrics import classification_report
print("Validation Set Classification Report:")
print(classification_report(y_val, val_predictions, target_names=['sadness', 'hopelessness', 'loneliness', 'anger', 'worthlessness', 'suicide intent', 'emptiness', 'brain dysfunction']))

# Predict on test set
test_predictions = trainer.predict(test_dataset).predictions.argmax(-1)

# Create submission
submission_df = pd.DataFrame({
    'sentence_id': test_df['sentence_id'],
    'predicted_emotion_int': test_predictions
})
submission_df.to_csv('submission_bert_imporveHo.csv', index=False)
print("Submission file created: submission_bert_imporveHo.csv")
files.download('submission_bert_imporveHo.csv')

Some weights of WeightedBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Macro F1
1,1.490800,1.134147,0.564120
2,0.885900,1.111297,0.602633
3,0.529600,1.395800,0.621667
4,0.325700,1.709105,0.643390
5,0.220800,2.032535,0.635645
6,0.181600,2.355385,0.634401
7,0.145100,2.611881,0.639844


Early stopping triggered after 7.0 epochs with no improvement in macro F1.


Validation Set Classification Report:
                   precision    recall  f1-score   support

          sadness       0.67      0.72      0.69      1422
     hopelessness       0.60      0.61      0.61       938
       loneliness       0.71      0.78      0.74       713
            anger       0.81      0.61      0.70       605
    worthlessness       0.56      0.57      0.56       305
   suicide intent       0.74      0.62      0.67       266
        emptiness       0.52      0.48      0.50       207
brain dysfunction       0.63      0.71      0.67       108

         accuracy                           0.66      4564
        macro avg       0.66      0.64      0.64      4564
     weighted avg       0.67      0.66      0.66      4564



Submission file created: submission_bert_imporveHo.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Patience 10**


In [17]:
import pandas as pd
import numpy as np
import torch
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers.trainer_callback import TrainerCallback, TrainerControl, TrainerState
from torch.utils.data import Dataset
from google.colab import files

# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Load training and test data
train_df = pd.read_csv('data/train_emotions.csv')
test_df = pd.read_csv('data/val_emotions_no_labels.csv')

# Combine title and sentence
train_df['text'] = train_df['title'].fillna('') + ' ' + train_df['sentence']
test_df['text'] = test_df['title'].fillna('') + ' ' + test_df['sentence']

# Prepare features and labels
X = train_df['text']
y = train_df['emotion_int']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Custom Dataset class
class EmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels if labels is not None else [0] * len(texts)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts.iloc[idx])
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        item = {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }
        if self.labels is not None and not isinstance(self.labels, list):
            item['labels'] = torch.tensor(self.labels.iloc[idx], dtype=torch.long)
        elif isinstance(self.labels, list):
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float).to('cuda')

# Custom model with weighted loss
class WeightedBertForSequenceClassification(BertForSequenceClassification):
    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = super().forward(input_ids, attention_mask=attention_mask, labels=labels)
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
            loss = loss_fct(outputs.logits, labels)
            return (loss, outputs.logits)
        return outputs

model = WeightedBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=8)

# Prepare datasets
train_dataset = EmotionDataset(X_train, y_train, tokenizer)
val_dataset = EmotionDataset(X_val, y_val, tokenizer)
test_dataset = EmotionDataset(test_df['text'], None, tokenizer)

# Custom callback for early stopping based on macro F1
class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, patience=10, threshold=0.0):
        self.patience = patience
        self.threshold = threshold
        self.best_macro_f1 = -float('inf')
        self.epochs_no_improve = 0

    def on_evaluate(self, args, state: TrainerState, control: TrainerControl, metrics, **kwargs):
        current_macro_f1 = metrics.get('eval_macro_f1', -float('inf'))
        if current_macro_f1 > self.best_macro_f1 + self.threshold:
            self.best_macro_f1 = current_macro_f1
            self.epochs_no_improve = 0
        else:
            self.epochs_no_improve += 1
            if self.epochs_no_improve >= self.patience:
                control.should_training_stop = True
                print(f"Early stopping triggered after {state.epoch} epochs with no improvement in macro F1.")

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    run_name='bert_emotion_classification',
    num_train_epochs=100,  # Set max epochs to 100
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='macro_f1',
    greater_is_better=True,
    logging_strategy='epoch',
    fp16=True,
    warmup_steps=500,
    weight_decay=0.1,
    learning_rate=2e-5
)

# Trainer with early stopping callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=lambda p: {'macro_f1': f1_score(p.label_ids, p.predictions.argmax(-1), average='macro')},
    callbacks=[EarlyStoppingCallback(patience=10, threshold=0.0)]  # Stop if no improvement for 3 epochs
)

# Train the model
trainer.train()

# Error analysis
val_predictions = trainer.predict(val_dataset).predictions.argmax(-1)
from sklearn.metrics import classification_report
print("Validation Set Classification Report:")
print(classification_report(y_val, val_predictions, target_names=['sadness', 'hopelessness', 'loneliness', 'anger', 'worthlessness', 'suicide intent', 'emptiness', 'brain dysfunction']))

# Predict on test set
test_predictions = trainer.predict(test_dataset).predictions.argmax(-1)

# Create submission
submission_df = pd.DataFrame({
    'sentence_id': test_df['sentence_id'],
    'predicted_emotion_int': test_predictions
})
submission_df.to_csv('submission_bert_imporveHo2.csv', index=False)
print("Submission file created: submission_bert_imporveHo2.csv")
files.download('submission_bert_imporveHo2.csv')

Some weights of WeightedBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Macro F1
1,1.490800,1.134147,0.564120
2,0.885900,1.111297,0.602633
3,0.529600,1.395800,0.621667
4,0.325700,1.709105,0.643390
5,0.220800,2.032535,0.635645
6,0.181600,2.355385,0.634401
7,0.145100,2.611881,0.639844
8,0.130300,2.758738,0.624398
9,0.105100,3.062620,0.622869
10,0.111000,2.982354,0.633603


Early stopping triggered after 14.0 epochs with no improvement in macro F1.


Validation Set Classification Report:
                   precision    recall  f1-score   support

          sadness       0.67      0.72      0.69      1422
     hopelessness       0.60      0.61      0.61       938
       loneliness       0.71      0.78      0.74       713
            anger       0.81      0.61      0.70       605
    worthlessness       0.56      0.57      0.56       305
   suicide intent       0.74      0.62      0.67       266
        emptiness       0.52      0.48      0.50       207
brain dysfunction       0.63      0.71      0.67       108

         accuracy                           0.66      4564
        macro avg       0.66      0.64      0.64      4564
     weighted avg       0.67      0.66      0.66      4564



Submission file created: submission_bert_imporveHo.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>